In [ ]:
from sqlalchemy.engine import create_engine
from sqlalchemy import inspect
from sqlalchemy import text
import csv

sqlite_engine = create_engine('sqlite:////path/mm_verified.db')
sqlite_connection = sqlite_engine.connect()

latestrun_courses = {
    "lgdm1x": {"title":"Leaders in Global Development", "link":'https://www.edx.org/course/leaders-global-development-uqx-lgdm1x-3', "instance_code":'UQx_LGDM1x_1T2018'},
    "lgdm2x": {"title":"The Science and Practice of Sustainable Development", "link":'https://www.edx.org/course/science-practice-sustainable-development-uqx-lgdm2x-2', "instance_code":'UQx_LGDM2x_1T2018'},
    "lgdm3x": {"title":"Adaptive Leadership in Development", "link":'https://www.edx.org/course/adaptive-leadership-development-uqx-lgdm3x-1', "instance_code":'UQx_LGDM3x_1T2018'},
    "lgdm4x": {"title":"Critical Development Perspectives",  "link":'https://www.edx.org/course/critical-development-perspectives-uqx-lgdm4x-1', "instance_code":'UQx_LGDM4x_1T2018'},
}

'''    
CREATE TABLE `dynamic_mailouts` (
	`id`	INTEGER PRIMARY KEY AUTOINCREMENT,
	`student_id`	INTEGER,
	`email`	TEXT,
	`no_enrolled_courses`	INTEGER,
	`trigger_course`	INTEGER,
	`trigger_course_verified`	INTEGER,
	`next_course_id`	INTEGER
);
'''

def db_refresh():
    sqlite_connection.execute("DELETE FROM dynamic_mailouts")
    
def db_insert (table, row):

    keys = row.keys();
    sql = 'INSERT INTO ' + table + ' ('
    sql = sql + ', '.join(keys)
    sql = sql + ') VALUES ('
    sql = sql + ', '.join([ ('"' + str(row[key]) + '"') for key in keys])
    sql = sql + ')'
    id = sqlite_connection.execute(text(sql));

    return id

def get_current_course_list(student_id):
    courseinstance_sql = "SELECT distinct course FROM course_instancedetails WHERE learner_id=%d AND course_instance LIKE '%2018%';" % (student_id)
    courseinstance_result = sqlite_connection.execute(courseinstance_sql);
    enrolled_courses = []
    for course in courseinstance_result:  
        enrolled_courses.append(course[0])
    #print(student_id)

    #print(enrolled_courses)
    return set(enrolled_courses)
    
        
db_refresh()
for course in latestrun_courses:
    current_instance = latestrun_courses[course]
    instance_code = current_instance['instance_code']
    instance_title = current_instance['title']
    instance_link = current_instance['link']
    courseinstance_sql = "SELECT * FROM course_instancedetails WHERE course_instance='%s' AND grade>=0.65;" % (instance_code)
    courseinstance_result = sqlite_connection.execute(courseinstance_sql);
    for student in courseinstance_result:
        student_id= student[1]
        enrollment_track = student[3]
        email = ""
        #get students email
        learner_sql = "SELECT email FROM learners WHERE student_id=%d;" % (student_id)
        learner_result = sqlite_connection.execute(learner_sql);
        for std in learner_result:
            email = std[0]
        trigger_course_verified = 0
        if enrollment_track == 'verified':
            trigger_course_verified = 1
        no_completedverified_courses = 0
        no_courses_sql = "SELECT no_completed_courses FROM lgdm_coursecompletion WHERE student_id=%d;" % (student_id)
        no_courses_sql_result = sqlite_connection.execute(no_courses_sql);
        for std2 in no_courses_sql_result:
            no_completedverified_courses = int(std2[0])
            
        #print(no_completedverified_courses)
        
        if no_completedverified_courses < 4:
            current_course_set = get_current_course_list(student_id)
            course_set = set(['lgdm1x','lgdm2x','lgdm3x','lgdm4x'])
            courses_not_enrolled_in = course_set.difference(current_course_set)
            courses_not_enrolled_in_list = sorted(list(courses_not_enrolled_in))
            #print(current_course_set)
            #print(courses_not_enrolled_in_list)
            if len(courses_not_enrolled_in_list)>0:
                next_course_id = courses_not_enrolled_in_list[0]

                next_course = latestrun_courses[next_course_id]
                #print(next_course)
                dynamic_mailouts_dict = {
                    'student_id':student_id,
                    'email':email,
                    'no_completedverified_courses':no_completedverified_courses,
                    'trigger_course':instance_code,
                    'trigger_course_verified':trigger_course_verified,
                    'trigger_course_title': instance_title,
                    'trigger_course_link': instance_link,
                    'next_course_link': next_course['link'], 
                    'next_course_title':next_course['title']
                }
                print(dynamic_mailouts_dict)
                #db_insert('dynamic_mailouts', dynamic_mailouts_dict)       
        
